In [5]:
from PIL import Image as img
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

from process_data import *
from model import *

In [11]:
import smtplib
from email.message import EmailMessage

def email_alert(subject, body , to):
    msg= EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    # msg['to'] = [to,"guddu.jijnasu999@gmail.com"]
    msg['to'] = to
    

    user = "jyotikrish669@gmail.com"#email account
    msg['from']= user
    password = "bemcgjrfmlzwztgo"#email id password
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    
    
    server.quit()
    
email_alert("mail subject","mail body","guddu.jijnasu999@gmail.com")

In [7]:

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")



def save_frame(video_path,save_dir,gap = 15,width=250,height=250):
    name = video_path.split("\\")[-1].split(".")[0]
    # name = video_path
    save_path = os.path.join(save_dir , name)
    create_dir(save_path)
    gun_path = os.path.join(save_dir , name, "detected_guns")
    create_dir(gun_path)
    cap=cv2.VideoCapture(video_path)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break
        if idx % gap ==0:#gap reduces the no. of frames per sec
            cv2.imwrite(f"{save_path}/{name}{idx}.jpg", frame)

            # ================================================================
            
            gun_img = img.open(f"{save_path}/{name}{idx}.jpg")
            # shp = np.array(gun_img).shape
            # if gun_img.height>=height and gun_img.width>=width and shp[-1]==3:
            gun_img = gun_img.resize((width,height))


            # gun_img = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
            gun = np.array(gun_img)
            gun = gun.flatten().reshape(width*height*3,1)
            # print(gun.T)
            
            theta = np.genfromtxt("theta.csv",delimiter=',').reshape(width*height*3+1,1)
            gun = np.vstack((gun,[[1]]))
            print(idx,predict(gun/255,theta))
            plt.close()
            plt.imshow(gun[:-1].reshape(width,height,3))
            plt.show()
            # ================================================================


        idx+=1


In [9]:
def extract_frames(videos_paths="captures",save_dir="frames",gap=15):
    all_videos = glob.glob(videos_paths+"/*")
    all_extracted = glob.glob(save_dir+"/*")
    for i,video in enumerate(all_extracted):
        all_extracted[i] = video.split("\\")[-1]
        
    for i,video in enumerate(all_videos):
        if video.split("\\")[-1].split(".")[0] in all_extracted:
            all_videos[i]=0
        
    
    while 0 in all_videos:
        all_videos.remove(0)
    # print(len(all_videos),all_videos)
    # print(len(all_extracted),all_extracted)

    for path in all_videos:
        print(path.split("\\")[-1])
        save_frame(path, save_dir,gap=gap)

In [10]:
extract_frames(gap=20)

Assassination attempt caught on CCTV.mp4
The Assassination.mp4
VID-20210630-WA0088.mp4
